In [11]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
import numpy as np 
import torch.nn.functional as F

In [12]:

model_name="meta-llama/Llama-3.2-1B-Instruct"
# Initialize the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [26]:
model.embed_tokens.norm_type

2.0